In [1]:
# Basic package
import pandas as pd
import numpy as np

#Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Save model for future use
import pickle

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

np.random.seed(42)

In [2]:
# Display all results in a cell, not just the last line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
df_1 =pd.read_csv('NFL Analytics Challenge Data Set 1.csv')
df_2 =pd.read_csv('NFL Analytics Challenge Data Set 2.csv')
df_3 =pd.read_csv('NFL Analytics Challenge Data Set 3.csv')

In [207]:
# Data cleaning and engineering
# Dataset 1
df_1['date'] =pd.to_datetime(df_1['date']) # Change to datetime format
df_1['away'] =df_1['away'].str.replace('.','').str.strip() # Remove punctuation and whitespace

# Convert text to number then split columns with composite number
df_1_cols_to_split =['third_downs_away','third_downs_home','fourth_downs_away','fourth_downs_home',
                     'comp_att_away','comp_att_home','sacks_away','sacks_home',
                     'penalties_away','penalties_home','redzone_away','redzone_home'
                    ]

dic ={'Jan':'1', 'Feb':'2', 'Mar':'3', 'Apr':'4', 'May':'5', 'Jun':'6',
      'Jul':'7', 'Aug':'8', 'Sep':'9', 'Oct':'10', 'Nov':'11', 'Dec':'12'
     }

for col in df_1_cols_to_split:
    df_1[col] =df_1[col].replace(dic, regex=True)
    df_1 =df_1.join(df_1[col].str.split('-', expand=True).add_prefix(col))
    
# Convert newly created columns to correct datatype to remove leading zeros
for col in df_1.columns[37:]:
    df_1[col] = df_1[col].astype('int64')
    
#df_1.info() #Validate that all columns have correct data type

# Convert two last columns to time in seconds
m =df_1['possession_away'].str.len().max()
df_1['possession_away'] =df_1['possession_away'].str.rjust(m, '0')
df_1['possession_home'] =df_1['possession_home'].str.rjust(m, '0')

df_1['possession_away_in_seconds'] =((df_1['possession_away'].str[:2].astype('int64'))*3600 +
                                            (df_1['possession_away'].str[3:5].astype('int64'))*60 +
                                            (df_1['possession_away'].str[6:].astype('int64')))

df_1['possession_home_in_seconds'] =((df_1['possession_home'].str[:2].astype('int64'))*3600 +
                                            (df_1['possession_home'].str[3:5].astype('int64'))*60 +
                                            (df_1['possession_home'].str[6:].astype('int64')))

In [211]:
# Data cleaning and engineering
# Dataset 2
df_2['date'] =pd.to_datetime(df_2['date']) # Change to datetime format
df_2['away'] =df_2['away'].str.replace('.','').str.strip() # Remove punctuation and whitespace

# Convert text to number then split columns with composite number
df_2_cols_to_split =['third_downs_away','third_downs_home','fourth_downs_away','fourth_downs_home',
                     'comp_att_away','comp_att_home','sacks_away','sacks_home',
                     'penalties_away','penalties_home','redzone_away','redzone_home'
                    ]

for col in df_2_cols_to_split:
    df_2[col] =df_2[col].replace(dic, regex=True)
    df_2 =df_2.join(df_2[col].str.split('-', expand=True).add_prefix(col))
    
# Convert newly created columns to correct datatype to remove leading zeros
for col in df_2.columns[37:]:
    df_2[col] = df_2[col].astype('int64')
    
#df_2.info() #Validate that all columns have correct data type

# Convert two last columns to time in seconds
m =df_2['possession_away'].str.len().max()
df_2['possession_away'] =df_2['possession_away'].str.rjust(m, '0')
df_2['possession_home'] =df_2['possession_home'].str.rjust(m, '0')

df_2['possession_away_in_seconds'] =((df_2['possession_away'].str[:2].astype('int64'))*3600 +
                                            (df_2['possession_away'].str[3:5].astype('int64'))*60 +
                                            (df_2['possession_away'].str[6:].astype('int64')))

df_2['possession_home_in_seconds'] =((df_2['possession_home'].str[:2].astype('int64'))*3600 +
                                            (df_2['possession_home'].str[3:5].astype('int64'))*60 +
                                            (df_2['possession_home'].str[6:].astype('int64')))

In [225]:
# Data cleaning and engineering
# Dataset 3
df_3['date'] =pd.to_datetime(df_3['date']) # Change to datetime format
df_3['away'] =df_3['away'].str.replace('.','').str.strip() # Remove punctuation and whitespace

In [230]:
# Create final dataframe
df = pd.concat([df_1, df_2],ignore_index=True)
df =df.merge(df_3, on=['date','away','home'])

In [241]:
df.info()
df.shape
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5088 entries, 0 to 5087
Data columns (total 65 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   date                        5088 non-null   datetime64[ns]
 1   away                        5088 non-null   object        
 2   home                        5088 non-null   object        
 3   first_downs_away            5088 non-null   int64         
 4   first_downs_home            5088 non-null   int64         
 5   third_downs_away            5088 non-null   object        
 6   third_downs_home            5088 non-null   object        
 7   fourth_downs_away           5088 non-null   object        
 8   fourth_downs_home           5088 non-null   object        
 9   passing_yards_away          5088 non-null   int64         
 10  passing_yards_home          5088 non-null   int64         
 11  rushing_yards_away          5088 non-null   int64       

(5088, 65)

,date,away,home,first_downs_away,first_downs_home,third_downs_away,third_downs_home,fourth_downs_away,fourth_downs_home,passing_yards_away,...,penalties_home0,penalties_home1,redzone_away0,redzone_away1,redzone_home0,redzone_home1,possession_away_in_seconds,possession_home_in_seconds,score_away,score_home
0,2002-09-05,49ers,Giants,13,21,12-4,16-9,0-0,0-1,166,...,10,80,0,8,0,6,99120,116880,16,13
1,2002-09-08,Seahawks,Raiders,14,27,11-1,12-7,2-2,1-1,143,...,5,45,0,2,0,2,90540,125460,17,31
2,2002-09-08,Jets,Bills,18,26,8-2,17-7,0-0,2-2,193,...,10,82,0,9,0,8,1266,140880,37,31
3,2002-09-08,Vikings,Bears,19,20,13-5,13-7,0-0,0-0,228,...,4,33,0,7,0,6,113460,102540,23,27
4,2002-09-08,Chargers,Bengals,27,13,10-6,11-4,0-0,0-0,160,...,9,57,0,7,0,5,136080,1332,34,6


In [246]:
# TO BE DISCUSSED

# Create classifier column, if away team win, then 1, if away team lose, then 0
df.loc[df['score_away'] > df['score_home'], 'away_win'] = 1 
df.loc[df['score_away'] < df['score_home'], 'away_win'] = 0 

# TO BE DETERMINED how to treat tie record?
#df['away_win'].unique()
#df[~df['away_win'].isin([0,1])] #11 tie record

In [259]:
df

,date,away,home,first_downs_away,first_downs_home,third_downs_away,third_downs_home,fourth_downs_away,fourth_downs_home,passing_yards_away,...,penalties_home1,redzone_away0,redzone_away1,redzone_home0,redzone_home1,possession_away_in_seconds,possession_home_in_seconds,score_away,score_home,away_win
0,2002-09-05,49ers,Giants,13,21,12-4,16-9,0-0,0-1,166,...,80,0,8,0,6,99120,116880,16,13,1.0
1,2002-09-08,Seahawks,Raiders,14,27,11-1,12-7,2-2,1-1,143,...,45,0,2,0,2,90540,125460,17,31,0.0
2,2002-09-08,Jets,Bills,18,26,8-2,17-7,0-0,2-2,193,...,82,0,9,0,8,1266,140880,37,31,1.0
3,2002-09-08,Vikings,Bears,19,20,13-5,13-7,0-0,0-0,228,...,33,0,7,0,6,113460,102540,23,27,0.0
4,2002-09-08,Chargers,Bengals,27,13,10-6,11-4,0-0,0-0,160,...,57,0,7,0,5,136080,1332,34,6,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5083,2022-01-23,Rams,Buccaneers,24,20,11-4,14-3,0-0,4-2,355,...,61,5,2,3,2,122880,93120,30,27,1.0
5084,2022-01-23,Bills,Chiefs,23,30,14-6,13-8,4-4,1-1,313,...,1,3,3,5,3,99420,131880,36,42,0.0
5085,2022-01-30,Bengals,Chiefs,21,24,14-8,12-6,0-0,0-0,243,...,2,4,1,5,3,129360,106920,27,24,1.0
5086,2022-01-30,49ers,Rams,16,25,9-3,18-11,0-0,0-1,232,...,2,1,1,6,2,87660,128340,17,20,0.0


In [261]:
# Only considered matches before 2022
df =df[df['date']<'2022-01-01']

# Only considered no tie matches
df =df[df['away_win'].isin([0,1])]

In [266]:
X = df.drop(columns=['date', 'away', 'home','third_downs_away', 'third_downs_home',
                     'fourth_downs_away', 'fourth_downs_home', 'comp_att_away', 'comp_att_home',
                     'sacks_away', 'sacks_home', 'penalties_away', 'penalties_home',
                     'redzone_away', 'redzone_home','possession_away','possession_home',
                     'away_win'])

y = df['away_win']